### 1. Создание признакового пространства 

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Это называется извлечением признаков.

##### Мешок слов
– это популярная и простая техника извлечения признаков, используемая при работе с текстом. Она описывает вхождения каждого слова в текст.

Чтобы использовать модель, нам нужно:

- Определить словарь известных слов (токенов).
- Выбрать степень присутствия известных слов.

Любая информация о порядке или структуре слов игнорируется. Вот почему это называется МЕШКОМ слов. Эта модель пытается понять, встречается ли знакомое слово в документе, но не знает, где именно оно встречается.

Интуиция подсказывает, что схожие документы имеют схожее содержимое. Также, благодаря содержимому, мы можем узнать кое-что о смысле документа.

Пример:
Рассмотрим шаги создания этой модели. Мы используем только 4 предложения, чтобы понять, как работает модель. В реальной жизни вы столкнетесь с бОльшими объемами данных.

In [ ]:
documents = ["I like this movie, it's it's it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']

Определяем словарь и создаем векторы документа. Соберем все уникальные слова из 4 загруженных предложений, игнорируя регистр, пунктуацию и односимвольные токены. Это и будет наш словарь (известные слова).

Для создания словаря можно использовать класс CountVectorizer из библиотеки sklearn.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
documents

["I like this movie, it's it's it's funny.",
 'I hate this movie.',
 'This was awesome! I like it.',
 'Nice one. I love it.']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer(ngram_range=(2, 2), analyzer='word', binary=True,)

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,awesome like,hate this,it funny,it it,like it,like this,love it,movie it,nice one,one love,this movie,this was,was awesome
0,0,0,1,1,0,1,0,1,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,1,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,1,0,1,1,0,0,0


Когда размер словаря увеличивается, вектор документа тоже растет. В примере выше, длина вектора равна количеству известных слов.

В некоторых случаях, у нас может быть неимоверно большой объем данных и тогда вектор может состоять из тысяч или миллионов элементов. Более того, каждый документ может содержать лишь малую часть слов из словаря.

Как следствие, в векторном представлении будет много нулей. Векторы с большим количеством нулей называются разреженным векторами (sparse vectors), они требуют больше памяти и вычислительных ресурсов. Частично эту проблему можно реить хорошей предобработкой.


##### Мешок N-грамм

Другой, более сложный способ создания словаря – использовать сгруппированные слова. Это изменит размер словаря и даст мешку слов больше деталей о документе. Такой подход называется «N-грамма».

N-грамма это последовательность каких-либо сущностей (слов, букв, чисел, цифр и т.д.). В контексте языковых корпусов, под N-граммой обычно понимают последовательность слов. Юниграмма это одно слово, биграмма это последовательность двух слов, триграмма – три слова и так далее. Цифра N обозначает, сколько сгруппированных слов входит в N-грамму. В модель попадают не все возможные N-граммы, а только те, что фигурируют в корпусе.

Пример:
Рассмотрим такое предложение:The office building is open today

Вот его биграммы:

- the office
- office building
- building is
- is open
- open today

Как видно, мешок биграмм – это более действенный подход, чем мешок слов.

In [ ]:
BPE
"I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."

nn - t1
uy - t2
t1u - t3

In [ ]:
from nltk.util import ngrams

text = "I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."
tokenized = text.split()
bigrams = ngrams(tokenized, 3)
list(bigrams)


[('I', 'like', 'this'),
 ('like', 'this', 'movie,'),
 ('this', 'movie,', "it's"),
 ('movie,', "it's", 'funny.'),
 ("it's", 'funny.', 'I'),
 ('funny.', 'I', 'hate'),
 ('I', 'hate', 'this'),
 ('hate', 'this', 'movie.'),
 ('this', 'movie.', 'This'),
 ('movie.', 'This', 'was'),
 ('This', 'was', 'awesome!'),
 ('was', 'awesome!', 'I'),
 ('awesome!', 'I', 'like'),
 ('I', 'like', 'it.'),
 ('like', 'it.', 'Nice'),
 ('it.', 'Nice', 'one.'),
 ('Nice', 'one.', 'I'),
 ('one.', 'I', 'love'),
 ('I', 'love', 'it.')]

##### TF-IDF

У частотного скоринга есть проблема: слова с наибольшей частотностью имеют, соответственно, наибольшую оценку. В этих словах может быть не так много информационного выигрыша для модели, как в менее частых словах. Один из способов исправить ситуацию – понижать оценку слова, которое часто встречается во всех схожих документах. Это называется TF-IDF.

TF-IDF (сокращение от term frequency — inverse document frequency) – это статистическая мера для оценки важности слова в документе, который является частью коллекции или корпуса.

Скоринг по TF-IDF растет пропорционально частоте появления слова в документе, но это компенсируется количеством документов, содержащих это слово.

<img src='image/tf_idf.PNG'>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

document = ["I like this movie, it's funny funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(document)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,awesome,funny,hate,it,like,love,movie,nice,one,this,was
0,0.000000,0.812578,0.000000,0.259329,0.320323,0.000000,0.320323,0.000000,0.000000,0.259329,0.000000
1,0.000000,0.000000,0.702035,0.000000,0.000000,0.000000,0.553492,0.000000,0.000000,0.448100,0.000000
2,0.539445,0.000000,0.000000,0.344321,0.425305,0.000000,0.000000,0.000000,0.000000,0.344321,0.539445
3,0.000000,0.000000,0.000000,0.345783,0.000000,0.541736,0.000000,0.541736,0.541736,0.000000,0.000000


In [ ]:
tfidf_vectorizer.idf_

array([1.91629073, 1.91629073, 1.91629073, 1.22314355, 1.51082562,
       1.91629073, 1.51082562, 1.91629073, 1.91629073, 1.22314355,
       1.91629073])

##### HashingVectorizer

Подсчеты и частоты могут быть очень полезны, но одним из ограничений этих методов является то, что словарный запас может стать очень большим. Это, в свою очередь, потребует больших векторов для кодирования документов и налагает большие требования к памяти и замедляет алгоритмы.

Умный обходной путь - использовать односторонний хэш слов, чтобы преобразовать их в целые числа. Умная часть заключается в том, что словарь не требуется, и вы можете выбрать произвольный длинный вектор фиксированной длины. Недостатком является то, что хеш является односторонней функцией, поэтому нет способа преобразовать кодировку обратно в слово (что может не иметь значения для многих контролируемых задач обучения).HashingVectorizer класс реализует этот подход, который можно использовать для последовательного хеширования слов, а затем для токенизации и кодирования документов по мере необходимости.

Пример ниже демонстрирует HashingVectorizer для кодирования одного документа.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

document = ["I like this movie, it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
vectorizer = HashingVectorizer(n_features=2**4,)
values = vectorizer.fit_transform(document)
print(values.shape)
print(values.toarray())

(4, 16)
[[ 0.37796447  0.          0.          0.          0.          0.
   0.         -0.37796447  0.          0.         -0.37796447  0.
   0.          0.          0.75592895  0.        ]
 [ 0.          0.          0.         -0.57735027  0.          0.
   0.          0.          0.          0.         -0.57735027  0.
   0.          0.          0.57735027  0.        ]
 [ 0.          0.4472136   0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.89442719  0.        ]
 [ 0.          0.          0.          0.         -0.5         0.
   0.          0.          0.         -0.5         0.          0.
   0.          0.5         0.5         0.        ]]


Выполнение примера кодирует образец документа как разреженный массив из 16 элементов. Значения закодированного документа соответствуют нормализованному количеству слов по умолчанию в диапазоне от -1 до 1, но могут быть сделаны простые целочисленные счетчики путем изменения конфигурации по умолчанию.

##### Для чего нужны Vectorizers?

Алгоритмы машинного обучения не могут напрямую работать с сырым текстом, поэтому необходимо конвертировать текст в наборы цифр (векторы). Уже с векторным представлением можно производить классификацию, к примеру.

In [ ]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [ ]:
from sklearn import model_selection, preprocessing, linear_model,
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count, train_y)
    predictions = classifier.predict(xvalid_count)
predictions

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(valid_y, predictionsictions)

0.8624

In [ ]:
trainDF['label'].value_counts()

__label__1    5097
__label__2    4903
Name: label, dtype: int64